In [1]:
import sys
!{sys.executable} -m pip install numpy tensorflow tf2onnx==1.15.0 onnx

  Using cached flatbuffers-23.5.26-py2.py3-none-any.whl (26 kB)
  Using cached tf2onnx-1.14.0-py3-none-any.whl (451 kB)
  Using cached tf2onnx-1.13.0-py3-none-any.whl (442 kB)
  Using cached tf2onnx-1.12.1-py3-none-any.whl (442 kB)
  Using cached tf2onnx-1.12.0-py3-none-any.whl (442 kB)
  Using cached tf2onnx-1.11.1-py3-none-any.whl (440 kB)
  Using cached tf2onnx-1.10.1-py3-none-any.whl (440 kB)
  Using cached tf2onnx-1.9.3-py3-none-any.whl (435 kB)
  Using cached tf2onnx-1.9.2-py3-none-any.whl (430 kB)
  Using cached tf2onnx-1.9.1-py3-none-any.whl (398 kB)
  Using cached tf2onnx-1.8.5-py3-none-any.whl (370 kB)
  Using cached tf2onnx-1.8.4-py3-none-any.whl (345 kB)
  Attempting uninstall: flatbuffers
    Found existing installation: flatbuffers 2.0.7
    Uninstalling flatbuffers-2.0.7:
      Successfully uninstalled flatbuffers-2.0.7
  Attempting uninstall: tf2onnx
    Found existing installation: tf2onnx 1.15.0
    Uninstalling tf2onnx-1.15.0:
      Successfully uninstalled tf2onnx-1

In [2]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

# Let's define mmodular MLP class:


class MLP(tf.keras.Model):
    def __init__(self, layer_sizes, operation_sequence):
        super(MLP, self).__init__()
        self.layers_list = []

        for i in range(len(layer_sizes) - 1):
            in_size, out_size = layer_sizes[i], layer_sizes[i+1]
            self.layers_list.extend(self._create_layer(
                in_size, out_size, operation_sequence))

    def _create_layer(self, in_features, out_features, operation_sequence):
        layer_components = []

        if 'linear_weights' in operation_sequence:
            layer_components.append(layers.Dense(out_features, use_bias=False))
        if 'linear_biases' in operation_sequence and 'linear_weights' not in operation_sequence:
            # If biases separately from weights, although it's non-standard
            layer_components.append(layers.Dense(
                out_features, use_bias=True, kernel_initializer='zeros'))
        if 'relu' in operation_sequence:
            layer_components.append(layers.ReLU())
        if 'scaling' in operation_sequence:
            layer_components.append(layers.Lambda(lambda x: x / 2.0))
        return layer_components

    def call(self, inputs):
        x = inputs
        for layer_component in self.layers_list:
            x = layer_component(x)
        return x

# Depth Benchmarks Example:

## Save model and convert to ONNX

In [3]:
model_name = 'mlp_499'

# Depth Benchmarks Example:
config = {
    'input_size': 200,
    'layer_size': 100,
    'num_hidden_size': 499,
    'output_size': 10
}

layer_sizes_depth = [config['input_size']] + [config['layer_size']] * config['num_hidden_size'] + [config['output_size']]
operation_sequence_depth = ['linear_weights', 'linear_biases', 'relu', 'scaling']

mlp_depth = MLP(layer_sizes_depth, operation_sequence_depth)

In [4]:
# -- Train model with random values --

import numpy as np

# Number of samples and features
num_samples = 10
num_features = 200
num_classes = 10

# Generate random data
x_train = np.random.randn(num_samples, num_features)

# Generate random labels (classes) for each sample
y_train = np.random.randint(num_classes, size=num_samples)

mlp_depth.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
mlp_depth.fit(x_train, y_train, epochs=5, batch_size=32)

# save model
model_path = f'./saved_models/{model_name}'
mlp_depth.save(model_path)

Epoch 1/5
1/1 [==============================] - 9s 9s/step - loss: 2.3026 - accuracy: 0.3000
Epoch 2/5
1/1 [==============================] - 0s 26ms/step - loss: 2.3026 - accuracy: 0.3000
Epoch 3/5
1/1 [==============================] - 0s 26ms/step - loss: 2.3026 - accuracy: 0.3000
Epoch 4/5
1/1 [==============================] - 0s 26ms/step - loss: 2.3026 - accuracy: 0.3000
Epoch 5/5
1/1 [==============================] - 0s 26ms/step - loss: 2.3026 - accuracy: 0.3000
INFO:tensorflow:Assets written to: ./saved_models/mlp_499/assets


INFO:tensorflow:Assets written to: ./saved_models/mlp_499/assets


In [5]:
# -- Convert trained model to ONNX --

import tf2onnx

# Parameters
onnx_path = f'./saved_models/{model_name}.onnx'    # desired path to save the ONNX model
input_signature = [tf.TensorSpec([None, 200], tf.float32, name="input")]

# Convert the model
model_proto, _ = tf2onnx.convert.from_keras(mlp_depth, input_signature=input_signature, output_path=onnx_path)

Instructions for updating:
This API was designed for TensorFlow v1. See https://www.tensorflow.org/guide/migrate for instructions on how to migrate your code to TensorFlow v2.


Instructions for updating:
This API was designed for TensorFlow v1. See https://www.tensorflow.org/guide/migrate for instructions on how to migrate your code to TensorFlow v2.


## Convert to Orion